In [1]:
import torch
import pandas as pd
import re
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments, BartConfig, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer
from datasets import Dataset


2024-04-25 22:05:03.576515: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-25 22:05:03.576620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-25 22:05:03.704698: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")
test = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")
validation = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

# Exploring the CNN DailyMail dataset 

>The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. 

>The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering.

- Abstractive: The text is summarized using the available context but using different words. This is a process that requires "understanding" the information contained in the text.

- Extractive: The most relevant phrases or words are selected and extracted from the text, without rephrasing or generating new words.

In [3]:
# to show the full article 
pd.set_option("display.max_colwidth", None)

In [4]:
train.loc[:, ["article", "highlights"]].sample(1)

,article,highlights
59815,"(CNN) -- Federal aid helped many cash-strapped Americans keep a roof over their heads during the prolonged economic slump, but the number of people living a step away from the streets has grown sharply, researchers reported Wednesday. The estimated U.S. homeless population dipped about 1% between 2009 and 2011 despite the lingering effects of the 2007-2009 recession, the Washington-based Homelessness Research Institute concluded. About $1.5 billion from the 2009 economic stimulus measure went toward rental assistance and programs steering recently evicted people toward new housing, ""and it seems likely that that has worked,"" said Nan Roman, president of the National Alliance to End Homelessness. Nevertheless, the homeless population went up in 24 states during the past two years, the Washington-based researchers found. And the number of people who have had to move in with friends or relatives -- often a last stop before the street -- is up as well, according to Wednesday's report. The Homelessness Research Institute, the educational arm of Roman's organization, put the number of Americans living on the streets or in shelters at just over 636,000 in 2011. That's down about 6,000 from the group's 2009 estimate. The figure is based on reports and street counts from state and local agencies that receive federal housing funds. Roman said the stimulus money, coupled with pre-recession federal programs aimed at veterans and the chronically ill, have kept that figure down even as the U.S. economy saw its worst downturn since the 1930s. But that money is drying up now that the Obama administration, Congress and the states are grappling with budget issues fueled by the recession. ""Just like all domestic discretionary spending, it's vulnerable at the moment because of the deficit,"" Roman said. States have already been cutting services, ""So again, it's notable that these are still working."" The report found the effects of the recession keep cascading downward more than two years after its official end in mid-2009. More than 4 million homes have been lost to foreclosure since 2007, according to the online marketer RealtyTrac. Those foreclosures have driven many ex-homeowners into the rental market, driving up rents and pushing some of the working poor to move in with family, friends or other acquaintances. That ""doubled-up"" population grew by about 800,000 people between 2009 and 2010, the last year data was available, the HRI report concluded. About 6.8 million were in combined households in 2010, a more than 50% increase from 2005, researchers found. ""The most common previous address for homeless people is that they were doubled up with someone else,"" Roman said. The new figures are ""not a good sign. It means we've got potentially a lot of people moving toward homelessness."" The number of low-income households that spend more than half their income on rent grew by 300,000 in 2009, to 6.2 million. That's about three-quarters of poor renters, the HRI study found. But the figures showed slight declines among nearly all categories examined, including a more than 10% decline in the number of unemployed veterans and about a 3% dip in the ranks of the chronically homeless -- people with disabilities like severe mental illness or addiction that result in repeated or extended stays on the street.","The estimated U.S. homeless population has remained basically flat, researchers say .\nBut a rise in people sharing housing is ""not a good sign,"" an advocate says .\nAbout $1.5 billion in stimulus money helped keep people in homes, she says ."


In [5]:
pd.reset_option('display.max_colwidth')

In [6]:
print(f"Train set shape {train.shape}")
print(f"Test set shape {test.shape}")
print(f"Validation set shape {validation.shape}")

Train set shape (287113, 3)
Test set shape (11490, 3)
Validation set shape (11490, 3)


In [7]:
# copy of the data
cpy_train = train.copy()

In [8]:
# average words in the article 
cpy_train["article_word_count"] = cpy_train["article"].apply(lambda x: len(x.split()))
average_article_word_counts = cpy_train["article_word_count"].mean()
print("Average words count in the Articles : ", average_article_word_counts)

Average words count in the Articles :  691.8694938926485


In [9]:
# sample 
cpy_train[["article_word_count"]].sample(10)

,article_word_count
25991,539
283957,400
261568,284
197188,452
149972,637
104616,479
245434,295
270154,1369
49296,309
49589,627


In [10]:
cpy_train[["article_word_count"]].describe()

,article_word_count
count,287113.000000
mean,691.869494
std,336.500035
min,8.000000
25%,443.000000
50%,632.000000
75%,877.000000
max,2347.000000


In [11]:
# average words in the summarization 
cpy_train["highlights_word_count"] = cpy_train["highlights"].apply(lambda x: len(x.split()))
average_highlights_word_counts = cpy_train["highlights_word_count"].mean()
print("Average words count in the highlights : ", average_highlights_word_counts)

Average words count in the highlights :  51.574101486174435


In [12]:
# sample
cpy_train[["highlights_word_count"]].sample(10)

,highlights_word_count
253693,50
233630,49
236765,44
93971,38
22865,44
198677,61
137040,50
186408,52
59310,44
62407,38


In [13]:
cpy_train[["highlights_word_count"]].describe()

,highlights_word_count
count,287113.000000
mean,51.574101
std,21.256336
min,4.000000
25%,38.000000
50%,48.000000
75%,60.000000
max,1296.000000


# Cleaning the text 

## HTML tags removing 

In [14]:
def detect_html_tags(text):
    if not isinstance(text, str):
        return []
    html_tags_patt = r"<[^>]+>"
    html_tags = re.findall(html_tags_patt, text)
    return html_tags

In [15]:
# Example text containing HTML tags
example_text = '''<p>This is a <b>sample</b> text with <a href='https://example.com'>HTML</a> tags.</p> <img src="/img/my-great-image.jpg" alt="My alt text" />'''

# Detect HTML tags in the text
html_tags = detect_html_tags(example_text)
html_tags

['<p>',
 '<b>',
 '</b>',
 "<a href='https://example.com'>",
 '</a>',
 '</p>',
 '<img src="/img/my-great-image.jpg" alt="My alt text" />']

In [16]:
cpy_train['article_html_tags'] = cpy_train['article'].apply(detect_html_tags)
cpy_train['highlights_html_tags'] = cpy_train['highlights'].apply(detect_html_tags)

In [17]:
cpy_train[['article_html_tags']].describe(include="O")

,article_html_tags
count,287113
unique,20
top,[]
freq,287093


In [18]:
# only one test sample has html tags
cpy_train[['highlights_html_tags']].describe(include="O")

,highlights_html_tags
count,287113
unique,2
top,[]
freq,287112


In [19]:
pd.set_option("max_colwidth", None)
display(cpy_train.loc[cpy_train['article_html_tags'].apply(len)>0, ["article_html_tags"]])
pd.reset_option('display.max_colwidth')

,article_html_tags
439,[<!--td {border: 1px solid #ccc;}br {mso-data-placement:same-cell;}-->]
6575,"[<fb:like href=""cnet.com"" font=""tahoma"">, </fb:like>]"
49099,[<Leaves his stupid wife forever>]
60098,"[<starts to click ""delete"">]"
73977,"[<iframe width=""640"" height=""360"" src=""//www.youtube.com/embed/LJlbDBJmavE"" frameborder=""0"" allowfullscreen>, </iframe>]"
76884,"[<!-- . /* Font Definitions */ . @font-face . {font-family:""ＭＳ 明朝""; . panose-1:0 0 0 0 0 0 0 0 0 0; . mso-font-charset:128; . mso-generic-font-family:roman; . mso-font-format:other; . mso-font-pitch:fixed; . mso-font-signature:1 134676480 16 0 131072 0;} . @font-face . {font-family:""Cambria Math""; . panose-1:2 4 5 3 5 4 6 3 2 4; . mso-font-charset:0; . mso-generic-font-family:auto; . mso-font-pitch:variable; . mso-font-signature:-536870145 1107305727 0 0 415 0;} . @font-face . {font-family:Cambria; . panose-1:2 4 5 3 5 4 6 3 2 4; . mso-font-charset:0; . mso-generic-font-family:auto; . mso-font-pitch:variable; . mso-font-signature:-536870145 1073743103 0 0 415 0;} . /* Style Definitions */ . p.MsoNormal, li.MsoNormal, div.MsoNormal . {mso-style-unhide:no; . mso-style-qformat:yes; . mso-style-parent:""""; . margin:0cm; . margin-bottom:.0001pt; . mso-pagination:widow-orphan; . font-size:12.0pt; . font-family:Cambria; . mso-ascii-font-family:Cambria; . mso-ascii-theme-font:minor-latin; . mso-fareast-font-family:""ＭＳ 明朝""; . mso-fareast-theme-font:minor-fareast; . mso-hansi-font-family:Cambria; . mso-hansi-theme-font:minor-latin; . mso-bidi-font-family:""Times New Roman""; . mso-bidi-theme-font:minor-bidi; . mso-ansi-language:EN-US;} . .MsoChpDefault . {mso-style-type:export-only; . mso-default-props:yes; . font-family:Cambria; . mso-ascii-font-family:Cambria; . mso-ascii-theme-font:minor-latin; . mso-fareast-font-family:""ＭＳ 明朝""; . mso-fareast-theme-font:minor-fareast; . mso-hansi-font-family:Cambria; . mso-hansi-theme-font:minor-latin; . mso-bidi-font-family:""Times New Roman""; . mso-bidi-theme-font:minor-bidi; . mso-ansi-language:EN-US;} . @page WordSection1 . {size:612.0pt 792.0pt; . margin:72.0pt 90.0pt 72.0pt 90.0pt; . mso-header-margin:36.0pt; . mso-footer-margin:36.0pt; . mso-paper-source:0;} . div.WordSection1 . {page:WordSection1;} . -->]"
111290,[<http://www.transforminglives.co.uk>]
112418,"[<string name=""paid_channel_subscribe_message"">, </string>, <string name=""paid_channel_unsubscribe_message"">, </string>]"
131864,"[<br/>, <font style=""font-size: 1.2em;"">, </font>, </p>, <div class=""clear"">, </div>, <div class=""artSplitter"">, <img id=""images_8763167_37"" src=""http://i.dailymail.co.uk/i/pix/2014/03/06/article-0-1C151D0700000578-736_634x610.jpg"" alt=""Vicious: A police dog (file picture) is far larger and more aggressive than a pug"" class=""articleImageEdit imageHeight PMS "" height=""610"" width=""634"">, <p class=""imageCaption"">, </p>, </div>, <p>, </p>, <p>, <font style=""font-size:1.2em;"">, </font>, </p>, <div class=""clear"">, </div>]"
163275,[<embed>]


In [20]:
# let's make a function to remove the html tags using re.sub()
# that replace a match with a string 
def remove_html_tags(text):
    # replace the tags with white space
    text = re.sub(r"<[^>]+>", "", text)
    # replace the white space it self
    # \ declare a usage of speacial sequnce 
    # \s (s small)Returns a match where the string contains a white space character
    # \s+ one or more
    # to remove the extra spaces we made
    text = re.sub(r"\s+", "", text)
    return text 


In [21]:
# let's apply this to the dataset 
filterd_indicies = cpy_train.loc[cpy_train['article_html_tags'].apply(len)>0].index
filterd_indicies
                                 

Index([   439,   6575,  49099,  60098,  73977,  76884, 111290, 112418, 131864,
       163275, 165768, 180412, 187295, 192013, 204861, 208063, 239766, 256114,
       260487, 272325],
      dtype='int64')

In [22]:
#  Apply remove_html_tags function to 'article' column
cpy_train.loc[filterd_indicies, 'article'] = cpy_train.loc[filterd_indicies, 'article'].apply(remove_html_tags)

In [23]:
# let's be sure
cpy_train['article_html_tags'] = cpy_train['article'].apply(detect_html_tags)
cpy_train[['article_html_tags']].describe(include="O")

,article_html_tags
count,287113
unique,1
top,[]
freq,287113


In [24]:
# now lets remove it from the highlights 
# yes there is a single observation that has html tags
# let's show it first 
cpy_train.loc[cpy_train["highlights_html_tags"].apply(len)>0,  ["highlights_html_tags"]]

,highlights_html_tags
189959,"[<font style=""font-size:1.4em;"">, <!-- .\n/* F..."


In [25]:
# lets see that first 
cpy_train.isnull().sum()

id                       0
article                  0
highlights               0
article_word_count       0
highlights_word_count    0
article_html_tags        0
highlights_html_tags     0
dtype: int64

In [26]:
train.isnull().sum()

id            0
article       0
highlights    0
dtype: int64

In [27]:
cpy_train.loc[cpy_train.isnull().sum()]

,id,article,highlights,article_word_count,highlights_word_count,article_html_tags,highlights_html_tags
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ...",198,37,[],[]


In [28]:
filterd_indicies_h = cpy_train.loc[cpy_train['highlights_html_tags'].apply(len)>0].index
cpy_train.loc[filterd_indicies_h, 'highlights'] = cpy_train.loc[filterd_indicies, 'highlights'].apply(remove_html_tags)


In [29]:
cpy_train['highlights_html_tags'] = cpy_train['highlights'].apply(detect_html_tags)
cpy_train[['highlights_html_tags']].describe(include="O")

,highlights_html_tags
count,287113
unique,1
top,[]
freq,287113


# Tokenization

In [30]:
# initialize Bart tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

# Example text
text = "The Quick brown fox jumps over the laze dog."

# Tokenize the text
tokenized_input = tokenizer(text, return_tensors="pt")

# Print tokenized input
print("Tokenized input:", tokenized_input)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Tokenized input: {'input_ids': tensor([[    0,   133, 13287,  6219, 23602, 13855,    81,     5,   784, 10129,
          2335,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
cpy_train_sample = cpy_train.sample(1000).copy()
cpy_train_sample

,id,article,highlights,article_word_count,highlights_word_count,article_html_tags,highlights_html_tags
212172,9ebcad42f1764ac4a508e0a2f0f6679fbbd4492d,(CNN) -- A violent crash on a dirt track in Ne...,NEW: NASCAR veteran Jason Leffler had just ret...,540,46,[],[]
198300,8cb12d549db8187037038c57f82ff027d5484eab,"MEXICO CITY, Mexico (CNN) -- Eight people wer...","NEW: Death toll rises to eight in two blasts, ...",399,35,[],[]
282029,f954b9cb727e14e555ffea8a77b58f5a4fb5ffa0,By . Daily Mail Reporter . PUBLISHED: . 12:46 ...,The jilted former First Lady jetted to the isl...,735,41,[],[]
224298,ae6f86df8675b0cee7e3e13886ad554637f835c1,(CNN) -- The mountain gorillas in a national p...,Census of mountain gorillas in DR Congo doing ...,206,38,[],[]
207488,98a8eff00e055acd92fccd5e96b4c725acebe8a3,WASHINGTON (CNN) -- Boys may soon be able to ...,Committee says shots could prevent genital war...,622,46,[],[]
...,...,...,...,...,...,...,...
197539,8bae94a23fc25e37b38cb925b33f3acc804e949c,"Meditating can make you warmer, researchers st...",Tibetan nuns increase their body temperatures ...,569,39,[],[]
276137,f1c54972119f429c980cbee713b7ff6d4e82b1d1,By . Ap . The first black woman to win an Olym...,Davis won a gold medal in the high jump in the...,604,54,[],[]
84011,ee4c77eb0cceef4b423f5fc8c7b032653dd0a77b,Louis van Gaal is expected to be appointed as ...,Van Gaal will want Manchester United move sort...,376,36,[],[]
224026,ae18f20b9b61f23fefe1a61346cef2f327e700aa,Three cops who posted a picture of their Portl...,The cops had posted a picture of their Portlan...,679,78,[],[]


In [32]:
data_sample = Dataset.from_pandas(cpy_train_sample.loc[:, ["article", "highlights"]])
data_sample

Dataset({
    features: ['article', 'highlights', '__index_level_0__'],
    num_rows: 1000
})

In [33]:
type(data_sample["highlights"])

list

In [34]:
# max_length = 1024

# tokenized_arts = tokenizer(data_sample["article"], padding="longest", truncation=True,max_length=max_length,return_tensors="pt" )
# tokenized_arts

In [35]:
# # tokenizer_2 = BartTokenizer.from_pretrained("facebook/bart-base", pad_token_id=-100)
# tokenizer_2 = BartTokenizer.from_pretrained("facebook/bart-base")

# special_tokens_dict = {'pad_token': '[MO]'}
# num_added_toks = tokenizer_2.add_special_tokens(special_tokens_dict)

# tokenized_high = tokenizer_2(data_sample["highlights"], padding="longest", truncation=True,max_length=max_length,return_tensors="pt" )
# tokenized_high

In [36]:
# tokenized_high['input_ids'][0]

In [37]:
# tokenizer_2.decode(tokenized_high['input_ids'][0])

In [38]:
# type(tokenized_high)

In [39]:
# type(tokenized_arts)

In [40]:
# tokenized_arts["input_ids"][0]

In [41]:
# tokenized_arts["attention_mask"][0]

In [42]:
# tokenized_high["input_ids"][0]

In [43]:
# tokenized_high["attention_mask"][0]

In [44]:
# tokenizer.get_added_vocab()

In [45]:
# tokenizer_2.get_added_vocab()

In [46]:
data_sample

Dataset({
    features: ['article', 'highlights', '__index_level_0__'],
    num_rows: 1000
})

In [47]:
def tokenizer_it(batch):
    encodings = tokenizer(batch["article"],text_target= batch["highlights"], padding="longest", truncation=True,max_length=1024,return_tensors="pt")
    return {
        'input_ids': encodings["input_ids"],
        'attention_mask':encodings["attention_mask"],
        'labels': encodings["labels"]
    }

In [48]:
data_tokenized = data_sample.map(tokenizer_it, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [49]:
data_tokenized

Dataset({
    features: ['article', 'highlights', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [50]:
print(data_tokenized["input_ids"][0])

[0, 1640, 16256, 43, 480, 83, 4153, 2058, 15, 10, 10667, 1349, 11, 188, 3123, 34, 1695, 5, 301, 9, 1015, 512, 1393, 3262, 1063, 3145, 1371, 6, 10, 10591, 3142, 18858, 30, 2598, 13816, 268, 25, 10, 11039, 8, 16106, 11439, 4, 1063, 3145, 1371, 6, 2908, 6, 962, 307, 6, 10, 410, 55, 87, 130, 377, 71, 3357, 7, 5, 232, 9, 765, 12, 16026, 490, 12, 13630, 4930, 71, 5650, 15, 10591, 28993, 13, 55, 87, 10, 2202, 6, 309, 7, 39, 998, 4, 22, 104, 9451, 259, 11, 26440, 4, 1666, 404, 38, 64, 206, 59, 16, 6127, 60, 10591, 28782, 7624, 10738, 1371, 26, 6, 5056, 7, 1063, 3145, 1371, 18, 195, 12, 180, 12, 279, 979, 4, 1063, 3145, 1371, 18, 744, 23, 4679, 3427, 13243, 11, 25517, 293, 9195, 6, 188, 3123, 6, 376, 148, 10, 7310, 1015, 148, 5, 4172, 9, 13770, 515, 6, 10, 564, 12, 22489, 1015, 13, 12631, 1677, 8895, 19, 12964, 2787, 11954, 4, 13780, 1677, 32, 239, 12, 10711, 1677, 14, 2333, 422, 15, 10667, 50, 21100, 31638, 5297, 4, 1063, 3145, 1371, 21, 15, 5, 887, 1004, 9, 5, 6104, 2859, 77, 39, 512, 314, 5,

In [51]:
print(tokenizer.decode(data_tokenized["input_ids"][0]))

<s>(CNN) -- A violent crash on a dirt track in New Jersey has claimed the life of race car driver Jason Leffler, a NASCAR veteran lauded by fellow racers as a fierce and versatile competitor. Leffler, 37, died Wednesday, a little more than three months after returning to the world of short-track open-wheel racing after focusing on NASCAR circuits for more than a decade, according to his website. "Sitting here in disbelief.... All I can think about is Charlie," NASCAR racer Elliott Sadler said, referring to Leffler's 5-year-old son. Leffler's death at Bridgeport Speedway in Swedesboro, New Jersey, came during a qualifying race during the Night of Wings event, a 25-lap race for sprint cars equipped with stabilizing wings. Sprint cars are high-powered cars that usually run on dirt or paved oval tracks. Leffler was on the fourth turn of the preliminary heat when his car left the banked dirt track and flipped several times down the front straightaway, the South Jersey Times newspaper report

In [52]:
print(data_tokenized["attention_mask"][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [53]:
print(tokenizer.decode(data_tokenized["labels"][0]))

<s>NEW: NASCAR veteran Jason Leffler had just returned to dirt-track racing.
He died Wednesday in a crash at a New Jersey dirt track.
NASCAR calls Leffler a "fierce competitor" who will be missed.
"Lost a good guy tonight," racer Bobby Labonte tweets.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [54]:
data_tokenized.format

{'type': None,
 'format_kwargs': {},
 'columns': ['article',
  'highlights',
  '__index_level_0__',
  'input_ids',
  'attention_mask',
  'labels'],
 'output_all_columns': False}

In [55]:
cols = ["input_ids", "labels", "attention_mask"]
data_tokenized.set_format(type="torch", columns=cols)

In [56]:
data_tokenized.format

{'type': 'torch',
 'format_kwargs': {},
 'columns': ['input_ids', 'labels', 'attention_mask'],
 'output_all_columns': False}

# Training Pipeline

In [57]:
type(data_tokenized)

datasets.arrow_dataset.Dataset

In [58]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
model

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): La

In [59]:
# collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

## Training Arguments 

In [60]:
# del cpy_train_sample
# del train
# del test 
# del validation

In [61]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model.to(device)
    
train_args = TrainingArguments(
    output_dir = "/kaggle/working/bart",
    num_train_epochs = 1,
    warmup_steps = 500,
    per_device_train_batch_size = 4,
    weight_decay = 0.01,
    logging_steps = 10,
    evaluation_strategy = 'steps',
    save_steps = 1e6,
    gradient_accumulation_steps=16
    
)


trainer = Trainer(model=model, args=train_args, tokenizer=tokenizer, data_collator=data_collator, train_dataset=data_tokenized, eval_dataset=data_tokenized)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()